In [2]:
pip install efficientnet_pytorch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install adamp

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=69ab4679a74f8cfe000ee445ba5cf1e06c51366aaeefa28a19c27534b023baa5
  Stored in directory: /opt/ml/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built adamp
Note: you may need to restart the kernel to use updated packages.


## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
from PIL import Image
from glob import glob
from tqdm import tqdm
import cv2
import numpy as np
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from efficientnet_pytorch import EfficientNet

from IPython.display import Audio

from adamp import AdamP

In [2]:
torch.cuda.empty_cache()

In [3]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [4]:
# EfficientNet b4
#model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18)
#model.cuda()

In [5]:
# Wide ResNet 50
# resnet = torchvision.models.wide_resnet50_2(pretrained=True)
# resnet.fc = torch.nn.Linear(2048, 18, True)
# model = resnet
# model.cuda()

In [6]:
# ResNet 152
resnet = torchvision.models.resnet152(pretrained=True)
resnet.fc = torch.nn.Linear(2048, 18, True)
model = resnet
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 2. Test Dataset 정의

In [7]:
train_dir = "/opt/ml/input/data/train/images"
test_dir = "/opt/ml/input/data/eval"
train_info = "/opt/ml/input/data/train/train.csv"

# data = pd.read_csv(train_info)
# data = data.sort_values("path")

# y = [] # 정답 레이블을 저장할 list

# data_per_class = [] # 각 class 별로 나눈 data들을 저장할 list

# for _ in range(18):
#     data_per_class.append([])
    
# for i in range(len(data)):
#     if data["gender"][i] == "male":
#         if data["age"][i] < 30:
#             # wear
#             data_per_class[0].append(i * 7 + 1)
#             data_per_class[0].append(i * 7 + 2)
#             data_per_class[0].append(i * 7 + 3)
#             data_per_class[0].append(i * 7 + 4)
#             data_per_class[0].append(i * 7 + 5)
#             y.append(0)
#             y.append(0)
#             y.append(0)
#             y.append(0)
#             y.append(0)
#             # incorrect
#             data_per_class[6].append(i * 7 + 0)
#             y.append(6)
#             # not wear
#             data_per_class[12].append(i * 7 + 6)
#             y.append(12)
#         elif data["age"][i] < 60:
#             # wear
#             data_per_class[1].append(i * 7 + 1)
#             data_per_class[1].append(i * 7 + 2)
#             data_per_class[1].append(i * 7 + 3)
#             data_per_class[1].append(i * 7 + 4)
#             data_per_class[1].append(i * 7 + 5)
#             y.append(1)
#             y.append(1)
#             y.append(1)
#             y.append(1)
#             y.append(1)
#             # incorrect
#             data_per_class[7].append(i * 7 + 0)
#             y.append(7)
#             # not wear
#             data_per_class[13].append(i * 7 + 6)
#             y.append(13)
#         else:
#             # wear
#             data_per_class[2].append(i * 7 + 1)
#             data_per_class[2].append(i * 7 + 2)
#             data_per_class[2].append(i * 7 + 3)
#             data_per_class[2].append(i * 7 + 4)
#             data_per_class[2].append(i * 7 + 5)
#             y.append(2)
#             y.append(2)
#             y.append(2)
#             y.append(2)
#             y.append(2)
#             # incorrect
#             data_per_class[8].append(i * 7 + 0)
#             y.append(8)
#             # not wear
#             data_per_class[14].append(i * 7 + 6)
#             y.append(14)
#     else:
#         if data["age"][i] < 30:
#             # wear
#             data_per_class[3].append(i * 7 + 1)
#             data_per_class[3].append(i * 7 + 2)
#             data_per_class[3].append(i * 7 + 3)
#             data_per_class[3].append(i * 7 + 4)
#             data_per_class[3].append(i * 7 + 5)
#             y.append(3)
#             y.append(3)
#             y.append(3)
#             y.append(3)
#             y.append(3)
#             # incorrect
#             data_per_class[9].append(i * 7 + 0)
#             y.append(9)
#             # not wear
#             data_per_class[15].append(i * 7 + 6)
#             y.append(15)
#         elif data["age"][i] < 60:
#             # wear
#             data_per_class[4].append(i * 7 + 1)
#             data_per_class[4].append(i * 7 + 2)
#             data_per_class[4].append(i * 7 + 3)
#             data_per_class[4].append(i * 7 + 4)
#             data_per_class[4].append(i * 7 + 5)
#             y.append(4)
#             y.append(4)
#             y.append(4)
#             y.append(4)
#             y.append(4)
#             # incorrect
#             data_per_class[10].append(i * 7 + 0)
#             y.append(10)
#             # not wear
#             data_per_class[16].append(i * 7 + 6)
#             y.append(16)
#         else:
#             # wear
#             data_per_class[5].append(i * 7 + 1)
#             data_per_class[5].append(i * 7 + 2)
#             data_per_class[5].append(i * 7 + 3)
#             data_per_class[5].append(i * 7 + 4)
#             data_per_class[5].append(i * 7 + 5)
#             y.append(5)
#             y.append(5)
#             y.append(5)
#             y.append(5)
#             y.append(5)
#             # incorrect
#             data_per_class[11].append(i * 7 + 0)
#             y.append(11)
#             # not wear
#             data_per_class[17].append(i * 7 + 6)
#             y.append(17)
            
# count_per_class = []
# for dl in data_per_class:
#     count_per_class.append(len(dl))
    
# print(count_per_class)
# print(sum(count_per_class))
# print(len(train_file_name))
# print(len(y))

In [8]:
# Dataset에서 사용할 class들
class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 60 else 2

In [9]:
class MaskDataset(Dataset):
    
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1.jpg": MaskLabels.mask,
        "mask2.jpg": MaskLabels.mask,
        "mask3.jpg": MaskLabels.mask,
        "mask4.jpg": MaskLabels.mask,
        "mask5.jpg": MaskLabels.mask,
        "incorrect_mask.jpg": MaskLabels.incorrect,
        "normal.jpg": MaskLabels.normal
    }
    
    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, transform = None, shuffle=True):
        self.img_dir = img_dir
        self.transform = transform

        self.setup()
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            for file_name, label in self._file_names.items():
                img_path = os.path.join(self.img_dir, profile, file_name)
                if os.path.exists(img_path):
                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    
                    if 57 <= int(age): # 57, 58, 59세를 모두 60세로 취급해서 학습 진행
                        age = "60"
                        
                    age_label = AgeGroup.map_label(age)
                    
                    self.image_paths.append(img_path)
                    self.mask_labels.append(label)
                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)
        
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = mask_label * 6 + gender_label * 3 + age_label

        if self.transform:
            image = self.transform(image)
            
        return image, multi_class_label
    
    def __len__(self):
        return len(self.image_paths)

In [10]:
class FocalLoss(nn.Module): # baseline code에 있던 FocalLoss()
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = torch.nn.functional.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return torch.nn.functional.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [11]:
transform = transforms.Compose([
    transforms.CenterCrop(350),
    transforms.Resize((256, 256)),
#     transforms.RandomApply([
#     transforms.RandomAffine((-5, 5))
# ], p=0.3),
    transforms.RandomApply([
    transforms.RandomRotation((-5, 5)),
], p=0.3),
    transforms.RandomHorizontalFlip(p = 0.5),
    ToTensor(),
    Normalize(mean=[0.548, 0.504, 0.479],
            std=[0.237, 0.247, 0.246]),
])

print(transform)
train_data = MaskDataset(train_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size = 64, num_workers = 4, shuffle=True)
test_data = MaskDataset(train_dir, transform=transform)
test_loader = DataLoader(test_data, batch_size = 8, num_workers = 2, shuffle=True)
criterion = FocalLoss()
optimizer = AdamP(model.parameters(), lr = 1e-4)

# for i, (img, label) in enumerate(tqdm(train_loader)):
#     if i == 10:
#         break

Compose(
    CenterCrop(size=(350, 350))
    Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
    RandomApply(
    p=0.3
    RandomRotation(degrees=(-5, 5), resample=False, expand=False)
)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246])
)


In [12]:
# augmentation_test_data = MaskDataset(train_dir, transform=transform)
# augmentation_test_loader = DataLoader(augmentation_test_data, shuffle=False)

# for img in augmentation_test_loader:
#     plt.imshow(transforms.ToPILImage(mode="RGB")(img[0].squeeze()))
#     plt.show()

In [13]:
# for iter, (img, label) in enumerate(train_loader):
#     if iter == 10: break
#     transforms.ToPILImage(imge)

In [14]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [15]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    transforms.CenterCrop(350),
    transforms.Resize((256, 256)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

def inference(i): # inference하고, submission.csv를 생성하는 function
    model.eval()

    all_predictions = []
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.cuda()
            pred = model(images)
            pred = torch.argmax(pred, 1)
            all_predictions.extend(pred.cpu().numpy())
    submission['ans'] = all_predictions

    # 제출할 파일을 저장합니다.
    submission.to_csv(os.path.join(test_dir, f'submission{i}.csv'), index=False)
    print('test inference is done!')
    Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg", autoplay=True)

In [16]:
min_loss = 9876543210

for _ in range(10):

    model.train()
    for iter, (img, label) in enumerate(tqdm(train_loader)):

        optimizer.zero_grad()
        img, label = img.float().cuda(), label.cuda()
        pred_logit = model.forward(img)
        loss = criterion(pred_logit, label)
        loss.backward()
        optimizer.step()
        
        if iter % 200 == 0: # batch를 200번 돌 때마다 validation -> 제대로 된 validation은 아님. 그냥 학습이 되고있는지 확인하는 정도
            total_len = 0
            total_cnt = 0
            model.eval()
            with torch.no_grad():
                for iter, (img, label) in enumerate(test_loader):

                    img, label = img.float().cuda(), label.cuda()
                    pred_logit = model.forward(img)
                    pred_label = torch.argmax(pred_logit, 1)
                    total_cnt += (pred_label == label).sum().item()
                    total_len += len(label)
                    break

                acc = total_cnt / total_len
                print(f"loss: {loss}, acc: {acc}, pred_label: {pred_label}")
        model.train()
        
    if min_loss > loss: # 이전 epoch보다 loss가 작아지면 inference해서 submission.csv 파일을 생성하도록 함
        min_loss = loss
        inference(_)

  0%|          | 1/564 [00:02<26:34,  2.83s/it]

loss: 2.647332191467285, acc: 0.0, pred_label: tensor([17, 17, 17, 17, 17, 17, 17, 17], device='cuda:0')


 36%|███▌      | 201/564 [04:00<07:51,  1.30s/it]

loss: 0.13358035683631897, acc: 0.75, pred_label: tensor([ 1,  9, 10,  5,  3,  1, 10,  4], device='cuda:0')


 71%|███████   | 401/564 [07:58<03:35,  1.32s/it]

...

loss: 0.014964388683438301, acc: 1.0, pred_label: tensor([15,  8,  1,  0,  3,  3,  5,  1], device='cuda:0')


  0%|          | 1/564 [00:02<23:15,  2.48s/it]

loss: 0.014710134826600552, acc: 1.0, pred_label: tensor([1, 3, 4, 3, 3, 0, 0, 1], device='cuda:0')


 34%|███▍      | 193/564 [03:50<07:22,  1.19s/it]


KeyboardInterrupt: 

## 3. Inference

In [26]:
!nvidia-smi

Thu Apr  1 00:16:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   35C    P0    56W / 250W |  24429MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  